<a href="https://colab.research.google.com/github/brepowell/ML-Contest-Series/blob/main/MLSeriesSupervisedLearningTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To run:
Use an IDE that will allow you to open a Jupyter Notebook.

For example, use Anaconda Navigator to open Visual Studio Code.

You may need to select a kernel to run the program.

# DATA GATHERING

## Import Libraries

In [6]:
# This step can be done in any cell of the notebook. It does not have to be at the top.
import pandas as pd


## Data Information

From:
https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households

More Info:
https://innovation.ukpowernetworks.co.uk/projects/low-carbon-london

"Created 9 years ago, updated 2 years ago
Energy consumption readings for a sample of 5,567 London Households that took part in the UK Power Networks led Low Carbon London project between November 2011 and February 2014.

Readings were taken at half hourly intervals. The customers in the trial were recruited as a balanced sample representative of the Greater London population.

The dataset contains energy consumption, in kWh (per half hour), unique household identifier, date and time. The CSV file is around 10GB when unzipped and contains around 167million rows.

Within the data set are two groups of customers. The first is a sub-group, of approximately 1100 customers, who were subjected to Dynamic Time of Use (dToU) energy prices throughout the 2013 calendar year period. The tariff prices were given a day ahead via the Smart Meter IHD (In Home Display) or text message to mobile phone. Customers were issued High (67.20p/kWh), Low (3.99p/kWh) or normal (11.76p/kWh) price signals and the times of day these applied. The dates/times and the price signal schedule is availaible as part of this dataset. All non-Time of Use customers were on a flat rate tariff of 14.228pence/kWh.

The signals given were designed to be representative of the types of signal that may be used in the future to manage both high renewable generation (supply following) operation and also test the potential to use high price signals to reduce stress on local distribution grids during periods of stress.

The remaining sample of approximately 4500 customers energy consumption readings were not subject to the dToU tariff."


## Create a huge dataframe with all hourly info

In [7]:
import pandas as pd
import glob

# Read all CSV files and concatenate them into a single DataFrame
path = "data/archive/hhblock_dataset/hhblock_dataset/" 

all_files = glob.glob(path + "block_*.csv")
df_list = []
for filename in all_files:
    df = pd.read_csv(filename)
    df_list.append(df)
full_df = pd.concat(df_list)


In [11]:
londonDataSortedByDay = full_df.sort_values(by='day')
londonDataSortedByDay.shape

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000155,2011-11-24,0.256,0.171,0.176,0.218,0.205,0.215,0.227,0.230,...,0.703,0.955,0.730,0.560,0.625,0.786,0.542,0.757,0.442,0.226
1605,MAC000151,2011-11-24,0.027,0.026,0.027,0.026,0.060,0.050,0.026,0.026,...,0.164,0.156,0.153,0.152,0.268,0.182,0.183,0.129,0.045,0.038
0,MAC000147,2011-11-24,0.271,0.176,0.118,0.115,0.114,0.111,0.055,0.048,...,0.269,0.495,0.469,0.444,0.390,0.293,0.358,0.171,0.299,0.180
2422,MAC000157,2011-11-24,0.388,0.428,0.323,0.334,0.330,0.175,0.186,0.253,...,0.292,0.324,0.261,0.230,0.316,0.290,0.258,0.349,0.340,0.295
824,MAC000153,2011-11-24,0.408,2.159,1.952,1.714,0.743,0.277,0.315,0.459,...,0.718,0.567,0.668,0.615,1.019,0.580,1.056,0.878,0.686,0.609


In [ ]:
londonDataSortedByDay.dropna(inplace=True)

## Visualize the features

In [ ]:
#https://www.tensorflow.org/tutorials/structured_data/time_series

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

df = block1
# Slice [start:stop:step], starting from index 5 take every 6th record.
df = df[5::17520]

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 5 to 1208885
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   LCLid           70 non-null     object
 1   tstp            70 non-null     object
 2   energy(kWh/hh)  70 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)

KeyError: 'tstp'

# STEP 2: FEATURE ENGINEERING / DATA CLEANING / PRE-PROCESSING TECHNIQUES

## Label your target variable

In [ ]:
# Ex) Change "yes" or "no" to 1 or 0 so the model understands the label

## Fix the data imbalance and other problems from STEP 1

## Seperate features (x) from labels (y)

## Feature Reduction

## Normalize / Scale the Data

## Investigate Variance or Feature Importance

# STEP 3: MODEL TRAINING & BUILDING


## Split into Train and Test data

## Use a Model

## Perform a Hyperparameter Search

## Save the best model

# STEP 4: EVALUATE THE MODEL

## Look at Metrics - Ex) Precision, Recall, F1 score

## Plot a Confusion Matrix

# STEP 5: DEPLOY MODEL